In [1]:
# Bibliotecas importadas
import pandas as pd
import re
import unicodedata
import binarytree
import avlTree
import gradio as gr
import time

In [2]:
# Carrega os CSVs em Dataframes
amazonReviews_df = pd.read_csv("Amazon Fine Food Reviews/Reviews.csv", usecols={"Summary", "Text"})
clickbaitHeadlines_df = pd.read_csv("Spam Clickbait News Headlines/examiner-date-text.csv", usecols={"headline_text"})
urbanDictionary_df = pd.read_csv("Urban Dictionary Words And Definitions/urbandict-word-defs.csv", usecols={"definition"})

In [3]:
def preprocessCorpus(corpus):
    # Une todas as strings da série em uma única string
    corpus = ' '.join(corpus.astype(str))

    # Remove caracteres não alfabéticos e acentos
    corpus = re.sub(r'[^a-zA-Z\s]', '', unicodedata.normalize('NFD', corpus))

    # Converte corpus em letras minúsculas e separa o corpus em uma lista
    corpus = corpus.lower().split()

    # Cria um Dataframe com o corpuso pré-processado
    corpus_df = pd.DataFrame({'Words': corpus})

    # Remove duplicatas do Dataframe
    corpus_df = corpus_df.drop_duplicates()

    # Retorna Dataframe
    return corpus_df

In [4]:
# Concatena todos os Dataframes
processCorpus_df = pd.DataFrame({'Words': pd.concat([preprocessCorpus(pd.concat([amazonReviews_df], axis=0).stack()), preprocessCorpus(pd.concat([clickbaitHeadlines_df], axis=0).stack()), preprocessCorpus(pd.concat([urbanDictionary_df], axis=0).stack())], axis=0).stack().reset_index(drop=True)})

# Exporta o DataFrame para CSV
processCorpus_df.to_csv("preprocessCorpus_df.csv", index=False)

# Head e tail do dataframe
processCorpus_df

,Words
0,good
1,quality
2,dog
3,food
4,i
...,...
1598034,uncloger
1598035,gigugle
1598036,identitarianism
1598037,likefollow


In [5]:
# Cria instâncias do dicionário e conjunto
wordCounts = {}
wordRepeated = set()

# Itera sobre cada palavra no DataFrame
for word in processCorpus_df['Words']:
    # Incrementa a contagem da palavra ou adiciona a palavra ao dicionário
    wordCounts[word] = wordCounts.get(word, 0) + 1

    # Verifica se a palavra atende ao critério de repetição
    if wordCounts[word] >= 3:
        wordRepeated.add(word)

# Substitui o Dataframe com as palavras repetidas
processCorpus_df = pd.DataFrame(wordRepeated, columns=['Words'])

# Remove duplicatas do Dataframe
processCorpus_df = processCorpus_df.drop_duplicates()

# Exporta o DataFrame para CSV
processCorpus_df.to_csv("processCorpus_df.csv", index=False)

# Head e tail do dataframe
processCorpus_df

,Words
0,gratis
1,surviving
2,crumpling
3,calmed
4,gran
...,...
57622,rigs
57623,tan
57624,hummer
57625,robbie


In [6]:
# Carrega o CSV em Dataframe para trabalhar sem pré-processamento
processCorpus_df = pd.read_csv("processCorpus_df.csv")

# Head e tail do dataframe
processCorpus_df

,Words
0,gratis
1,surviving
2,crumpling
3,calmed
4,gran
...,...
57622,rigs
57623,tan
57624,hummer
57625,robbie


In [7]:
# Função para criação da Árvore Binária
def bstBuild(root, word):
    if root is None:
        return binarytree.Node(word)
    if str(word) < str(root.value):
        root.left = bstBuild(root.left, word)
    elif str(word) > str(root.value):
        root.right = bstBuild(root.right, word)
    return root

# Carregando Dataframe em BST
root = None
for word in processCorpus_df['Words'].head(10):
    root = bstBuild(root, word)

# Amostra de 10 elementos da BST
print(root)


             _________gratis_____________________
            /                                    \
    ___crumpling_                         ____surviving____________
   /             \                       /                         \
calmed           gran              _rhodesian                ___underused
                                  /                         /
                                _nx                     twoword
                               /
                             imo



In [8]:
# Criando uma instância da árvore AVL
avl = avlTree.AvlTree()

# Carregando Dataframe em Árvore AVL
for words in processCorpus_df['Words']:
    avl.add(str(words))

# Altura da árvore AVL
print("Altura da árvore AVL:", avl.get_height())

Altura da árvore AVL: 19


In [9]:
def wordMatch(tree, characters):
    # Verifique se a entrada tem pelo menos dois caracteres
    if len(characters) < 2:
        return []

    # Pesquisa na árvore e adiciona nós correspondentes
    def searchRecursive(node, matches):
        if node is None:
            return

        # Adicione a letra do nó atual à sequência atual
        currentChars = node.value

        # Verifique se a sequência de caracteres fornecida está em qualquer lugar da palavra do nó atual
        if currentChars.startswith(characters):
            matches.append(currentChars)

        # Recurso para o nó esquerdo
        searchRecursive(node.left_child, matches)

        # Recurso para o nó direito
        searchRecursive(node.right_child, matches)

    # Inicialize a lista de correspondências
    matches = []

    # Chame a função de pesquisa recursiva a partir da raiz da árvore
    searchRecursive(tree.root, matches)

    return matches

In [10]:
# Loop principal para a buscar de palavras por letras
while True:
  # Input externo
  inputText = input("Buscar: ")

  # Verifica se o usuário deseja sair do loop.
  if inputText == "exit":
    break

  # Registra o tempo de início da busca.
  startTime = time.time()

  # Realiza a busca das palavras na árvore AVL.
  avlMatch = wordMatch(avl, inputText)

  # Calcula o tempo total de busca.
  searchTime = time.time() - startTime

  # Exibe os resultados da busca.
  for match in avlMatch:
    print(match)

  # Exibe o tempo total de busca.
  print(f"Tempo de busca: {searchTime:.6f} segundos")

gamestop
gameday
gameboy
game
gamechanger
gamed
gamer
gamelike
gamely
games
gamers
gamesa
gamey
gametype
Tempo de busca: 0.038581 segundos
poked
poke
pokey
poker
pokemon
pokes
Tempo de busca: 0.044429 segundos


In [11]:
# Função de recursão para busca em AVL
def prefix_search(prefix):
    words = wordMatch(avl, prefix)
    return words if words else "Nenhuma palavra encontrada com este prefixo"

# Criação de aplicação web para buscar palavras por prefixo
interface = gr.Interface(
    fn=prefix_search,
    inputs="text",
    outputs="text",
    title="Busca de palavras por prefixo",
    description="Digite um prefixo para encontrar palavras no corpus que tenham a mesma inicial",
)

# Carregamento da interface
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
